In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urljoin, urlparse
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def setup_driver():
    """Chrome 드라이버 설정"""
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # 브라우저 창 숨기기
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def download_single_episode(title, episode_url):
    """
    단일 회차의 웹툰 이미지를 다운로드하는 함수 (Selenium 사용)
    """
    driver = None
    try:
        # 드라이버 설정
        driver = setup_driver()
        
        # 페이지 로드
        driver.get(episode_url)
        
        # JavaScript 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "wt_viewer"))
        )
        
        # 회차 번호 추출 (URL에서)
        episode_no = re.search(r'no=(\d+)', episode_url)
        if episode_no:
            episode_no = episode_no.group(1)
        else:
            episode_no = "unknown"
        
        print(f"다운로드 중: {title} - {episode_no}화")
        
        # 웹툰 이미지들 찾기
        img_elements = driver.find_elements(By.CSS_SELECTOR, ".wt_viewer img")
        
        if not img_elements:
            print(f"이미지를 찾을 수 없습니다: {episode_no}화")
            return False
        
        # 폴더 생성
        folder_path = os.path.join('img', title, episode_no)
        os.makedirs(folder_path, exist_ok=True)
        
        # 헤더 설정
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': episode_url
        }
        
        # 각 이미지 다운로드
        for i, img_element in enumerate(img_elements, 1):
            img_url = img_element.get_attribute('src')
            if not img_url:
                continue
            
            # 상대 URL to 절대 URL
            if img_url.startswith('//'):
                img_url = 'https:' + img_url
            elif img_url.startswith('/'):
                img_url = 'https://comic.naver.com' + img_url
            
            try:
                # 이미지 다운로드
                img_response = requests.get(img_url, headers=headers)
                img_response.raise_for_status()
                
                # 파일명 생성
                file_extension = os.path.splitext(urlparse(img_url).path)[1] or '.jpg'
                filename = f"{i:03d}{file_extension}"
                file_path = os.path.join(folder_path, filename)
                
                # 파일 저장
                with open(file_path, 'wb') as f:
                    f.write(img_response.content)
                
                print(f"  └ 저장: {filename}")
                
            except Exception as e:
                print(f"  └ 이미지 다운로드 실패: {img_url} - {e}")
        
        return True
        
    except Exception as e:
        print(f"회차 다운로드 실패: {episode_url} - {e}")
        return False
    finally:
        if driver:
            driver.quit()

def download_all_episode(title, episode_url):
    """
    하나의 웹툰의 여러 회차 이미지를 다운로드하는 함수 (Selenium 사용)
    
    Args:
        title (str): 웹툰 제목
        episode_url (str): 웹툰 회차 리스트 페이지 URL
    """
    driver = None
    try:
        print(f"웹툰 '{title}' 회차 목록을 가져오는 중...")
        
        # 드라이버 설정
        driver = setup_driver()
        
        # 페이지 로드
        driver.get(episode_url)
        
        # JavaScript 로딩 대기 (회차 리스트가 로드될 때까지)
        time.sleep(5)
        
        # 회차 링크들 찾기 (다양한 셀렉터 시도)
        episode_links = []
        
        # 방법 1: viewList 테이블에서 찾기
        try:
            link_elements = driver.find_elements(By.CSS_SELECTOR, "table.viewList td.title a")
            if link_elements:
                print(f"viewList 테이블에서 {len(link_elements)}개 링크 발견")
                for element in link_elements:
                    href = element.get_attribute('href')
                    if href:
                        episode_links.append(href)
        except:
            pass
        
        # 방법 2: 일반적인 회차 링크 패턴으로 찾기
        if not episode_links:
            try:
                link_elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='/webtoon/detail']")
                print(f"일반 패턴에서 {len(link_elements)}개 링크 발견")
                for element in link_elements:
                    href = element.get_attribute('href')
                    if href and 'titleId=823933' in href:
                        episode_links.append(href)
            except:
                pass
        
        # 방법 3: 모든 링크에서 패턴 매칭
        if not episode_links:
            try:
                all_links = driver.find_elements(By.TAG_NAME, "a")
                print(f"전체 {len(all_links)}개 링크 검사 중...")
                for element in all_links:
                    href = element.get_attribute('href')
                    if href and '/webtoon/detail' in href and 'titleId=823933' in href:
                        episode_links.append(href)
            except:
                pass
        
        # 중복 제거
        episode_links = list(dict.fromkeys(episode_links))
        
        if not episode_links:
            print("회차 링크를 찾을 수 없습니다.")
            print("페이지 소스 일부:")
            print(driver.page_source[:2000])
            return
        
        print(f"총 {len(episode_links)}개의 회차를 찾았습니다.")
        
        # 각 회차 다운로드
        success_count = 0
        for i, link in enumerate(episode_links, 1):
            print(f"\n[{i}/{len(episode_links)}] 다운로드 진행 중...")
            
            if download_single_episode(title, link):
                success_count += 1
            
            # 서버 부하 방지를 위한 딜레이
            time.sleep(2)
        
        print(f"\n다운로드 완료: {success_count}/{len(episode_links)}개 회차 성공")
        
    except Exception as e:
        print(f"전체 다운로드 실패: {e}")
    finally:
        if driver:
            driver.quit()

# 사용 예시
if __name__ == "__main__":
    # 배달왕 웹툰 다운로드
    download_all_episode('배달왕', 'https://comic.naver.com/webtoon/list?titleId=823933')

웹툰 '배달왕' 회차 목록을 가져오는 중...
일반 패턴에서 21개 링크 발견
총 21개의 회차를 찾았습니다.

[1/21] 다운로드 진행 중...
